## Tratamento do PPA por regionalização

Nesse notebook são tratados os dados do Plano Plurianual por regionalização (2022 - 2025) da Prefeitura Municipal de São Paulo

In [1]:
import pandas as pd
from utils.load_csv import load_csv
from utils.save_csv import save_csv

In [2]:
df_ppa_reg = load_csv("ppa_reg.csv")

#### Criação "id"

Cria variável "id" concatenando variáveis de código da base de dados para "chave" do merge

In [3]:
df_ppa_reg["id"] = df_ppa_reg[["codigo orgao",
                               "codigo unidade",
                               "codigo programa",
                               "codigo acao"]].apply(lambda x: "_".join(x.astype(str)), axis = 1)

In [4]:
cols = ["id"] + [col for col in df_ppa_reg.columns if col != "id"]

df_ppa_reg = df_ppa_reg[cols]

#### Transformação das variáveis de planejamento de recurso

Transforma as variáveis de planejamento de resurso da base de dados de string para float

In [5]:
cols_plan_recurso = ["valor 2022",
                     "valor 2023",
                     "valor 2024",
                     "valor 2025"]

In [6]:
for col in cols_plan_recurso:
    df_ppa_reg[col] = (df_ppa_reg[col]
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .astype(float))

#### Filtro regionalizável

Filtra a base de dados para conter apenas despesas regionalizáveis

In [7]:
supra_subprefs = ["Supra Subprefeitura",
                  "Supra Subprefeitura Leste",
                  "Supra Subprefeitura Oeste",
                  "Supra Subprefeitura Norte",
                  "Supra Subprefeitura Sul",
                  "Supra Subprefeitura Centro"]

In [8]:
df_ppa_reg = df_ppa_reg[~df_ppa_reg["descricao prefeitura regional"].isin(supra_subprefs)]

#### Normalização das subprefeituras

Trata os nomes das subprefreituras na base de dados para estar igual aos nomes do shapefile das subprefeituras

In [9]:
mapper = {"Subprefeitura Sé": "SE",
          "Subprefeitura Vila Mariana": "VILA MARIANA",
          "Subprefeitura Santana/Tucuruvi": "SANTANA-TUCURUVI",
          "Subprefeitura Campo Limpo": "CAMPO LIMPO",
          "Subprefeitura Itaim Paulista": "ITAIM PAULISTA",
          "Subprefeitura Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
          "Subprefeitura Sapopemba": "SAPOPEMBA",
          "Subprefeitura Capela do Socorro": "CAPELA DO SOCORRO",
          "Subprefeitura Lapa": "LAPA",
          "Subprefeitura Jaçanã/Tremembé": "JACANA-TREMEMBE",
          "Subprefeitura Penha": "PENHA",
          "Subprefeitura São Miguel Paulista": "SAO MIGUEL",
          "Subprefeitura Perus/Anhanguera": "PERUS",
          "Subprefeitura Butantã": "BUTANTA",
          "Subprefeitura de Guaianases": "GUAIANASES",
          "Subprefeitura São Mateus": "SAO MATEUS",
          "Subprefeitura Cidade Ademar": "CIDADE ADEMAR",
          "Subprefeitura de Vila Prudente": "VILA PRUDENTE",
          "Subprefeitura Ipiranga": "IPIRANGA",
          "Subprefeitura Pinheiros": "PINHEIROS",
          "Subprefeitura Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
          "Subprefeitura Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
          "Subprefeitura M'Boi Mirim": "M BOI MIRIM",
          "Subprefeitura Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
          "Subprefeitura Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
          "Subprefeitura Santo Amaro": "SANTO AMARO",
          "Subprefeitura Mooca": "MOOCA",
          "Subprefeitura Ermelino Matarazzo": "ERMELINO MATARAZZO",
          "Subprefeitura Cidade Tiradentes": "CIDADE TIRADENTES",
          "Subprefeitura Itaquera": "ITAQUERA",
          "Subprefeitura Jabaquara": "JABAQUARA",
          "Subprefeitura Parelheiros": "PARELHEIROS"}

In [10]:
df_ppa_reg.loc[:, "descricao prefeitura regional"] = df_ppa_reg["descricao prefeitura regional"].map(mapper)

#### Salvar

In [11]:
pd.set_option("display.max_columns", None)

df_ppa_reg.sample(10)

,id,codigo orgao,descricao orgao,codigo unidade,descricao unidade,codigo programa,descricao programa,codigo acao,descricao acao,codigo da,descricao da,codigo meta,descricao meta,codigo regiao,descricao regiao,codigo prefeitura regional,descricao prefeitura regional,codigo distrito,descricao distrito,codigo sequencial da,codigo produto,descricao produto,descricao unidade medida,quantidade 2022,valor 2022,quantidade 2023,valor 2023,quantidade 2024,valor 2024,quantidade 2025,valor 2025
8838,87_10_3009_4656,87,Fundo Municipal de Desenvolvimento de Trânsito,10,Fundo Municipal de Desenvolvimento de Trânsito,3009,Melhoria da Mobilidade Urbana Universal,4656,Manutenção e Operação do Policiamento de Trânsito,777.04.19.00.001,Subprefeitura Capela do Socorro,777,Despesa Regionalizável,4,Sul,19,CAPELA DO SOCORRO,0,Supra-Distrital,1,2977,Operação/Serviço mantido,unida,"73.844,00",3110301.0,"74.046,00",3118822.0,74.185,3124653.0,74.335,3130990.0
239,12_10_3005_2705,12,Secretaria Municipal das Subprefeituras,10,Gabinete do Secretário,3005,Promoção da Sustentabilidade Ambiental,2705,Manutenção e Operação de Áreas Verdes e Vegeta...,777.01.02.00.001,Subprefeitura Pirituba/Jaraguá,777,Despesa Regionalizável,1,Norte,2,PIRITUBA-JARAGUA,0,Supra-Distrital,1,2571,Quantidade de manejo arbóreo realizado,unida,"0,00",1708000.0,"0,00",1278876.0,0,1398656.0,0,1519534.0
6412,29_40_3002_3354,29,Secretaria Municipal de Urbanismo e Licenciamento,40,Operação Urbana Centro,3002,Acesso à Moradia Adequada,3354,Construção de Unidades Habitacionais,777.05.09.00.001,Subprefeitura Sé,777,Despesa Regionalizável,5,Centro,9,SE,0,Supra-Distrital,1,2648,Unidades Habitacionais entregues por meio de p...,unida,"0,00",432649.0,"1,00",477290.0,1,681416.0,1,681416.0
5206,25_10_3001_6388,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6388,Rádios Comunitárias - Lei nº 16.572/2016,777.02.21.00.001,Subprefeitura Penha,777,Despesa Regionalizável,2,Leste,21,PENHA,0,Supra-Distrital,1,2898,Proponentes com projetos fomentados e/ou incen...,propo,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
4785,25_10_3001_6377,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,3001,Acesso à Cultura,6377,Programa de Gestão Cultural Comunitária de Esp...,777.01.01.00.001,Subprefeitura Perus/Anhanguera,777,Despesa Regionalizável,1,Norte,1,PERUS,0,Supra-Distrital,1,2910,Ocupações Culturais participantes,parti,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
6851,34_10_3018_2051,34,Secretaria Municipal de Direitos Humanos e Cid...,10,Gabinete do Secretário,3018,"Promoção da Cidadania, Inclusão Digital e Valo...",2051,Manutenção e Operação de Equipamentos Públicos...,777.03.10.00.001,Subprefeitura Butantã,777,Despesa Regionalizável,3,Oeste,10,BUTANTA,0,Supra-Distrital,1,2778,Pessoas atendidas pelo Centro de Referência e ...,pesso,"1,00",62371.0,"1,00",65331.0,1,68819.0,1,72973.0
6021,27_10_3005_2704,27,Secretaria Municipal do Verde e do Meio Ambiente,10,Gabinete do Secretário,3005,Promoção da Sustentabilidade Ambiental,2704,Manutenção e Operação dos Planetários Municipais,777.04.14.00.001,Subprefeitura Santo Amaro,777,Despesa Regionalizável,4,Sul,14,SANTO AMARO,0,Supra-Distrital,1,2825,Atividades desenvolvidas pelo planetário do Carmo,ações,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
1014,15_10_3010_2872,15,Cinema e Audiovisual de São Paulo,10,Cinema e Audiovisual de São Paulo,3010,"Manutenção, Desenvolvimento e Promoção da Qual...",2872,"Eventos Educacionais, Culturais e Esportivos n...",777.04.16.00.001,Subprefeitura Cidade Ademar,777,Despesa Regionalizável,4,Sul,16,CIDADE ADEMAR,0,Supra-Distrital,1,2843,"Participantes em eventos educacionais, cultura...",pesso,"0,00",0.0,"0,00",0.0,0,0.0,0,0.0
7776,75_10_3005_1702,75,Fundo Municipal de Parques,10,Fundo Municipal de Parques,3005,Promoção da Sustentabilidade Ambiental,1702,Construção e Implantação de Parques Urbanos e ...,777.02.24.00.001,Subprefeitura Itaim Paulista,777,Despesa Regionalizável,2,Leste,24,ITAIM PAULISTA,0,Supra-Distrital,1,2766,No

In [12]:
save_csv(df_ppa_reg, "ppa_reg.csv")

Base salva em data\ppa_reg.csv
